In [34]:
import os
from openai import AzureOpenAI
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
import re


# --- 1. Configuración del Cliente de Azure OpenAI ---


In [35]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
deployment_name = "gpt-4o"

# --- 2. Definición del Estado del Subgrafo ---
# Este diccionario define la "memoria" que se comparte entre los nodos del subgrafo.

In [36]:
class SubGraphState(TypedDict):
    user_query: str
    sql_query: str
    sql_results: str
    final_analysis: str

# --- 2. Datos de Entrada Fijos ---

In [37]:
fixed_sql_query = """
SELECT
    dc.customer_key,
    dc.first_name || ' ' || dc.last_name AS cliente,
    COUNT(DISTINCT fis.sales_order_number) AS pedidos_totales,
    COUNT(DISTINCT CASE WHEN dpromo.promotion_key IS NOT NULL THEN fis.sales_order_number END) AS pedidos_con_promo,
    ROUND(
        100.0 * COUNT(DISTINCT CASE WHEN dpromo.promotion_key IS NOT NULL THEN fis.sales_order_number END) 
        / NULLIF(COUNT(DISTINCT fis.sales_order_number), 0),
        2
    ) AS tasa_conversion_promocion_pct
FROM adventure_works.fact_internet_sales fis
JOIN adventure_works.dim_customer dc
    ON fis.customer_key = dc.customer_key
LEFT JOIN adventure_works.dim_promotion dpromo
    ON fis.promotion_key = dpromo.promotion_key
WHERE fis.order_date BETWEEN '2004-01-01' AND '2020-12-31'
GROUP BY dc.customer_key, cliente
HAVING COUNT(DISTINCT fis.sales_order_number) >= 5
ORDER BY tasa_conversion_promocion_pct DESC
LIMIT 20;
"""

fixed_sql_results = """customer_key	cliente	pedidos_totales	pedidos_con_promo	tasa_conversion_promocion_pct
11078	Gina Martin	17	17	100.00
11091	Dalton Perez	28	28	100.00
11131	Amanda Rivera	5	5	100.00
11142	Eduardo Patterson	17	17	100.00
11176	Mason Roberts	28	28	100.00
11185	Ashley Henderson	27	27	100.00
11200	Jason Griffin	27	27	100.00
11203	Luis Diaz	17	17	100.00
11211	Samantha Russell	17	17	100.00
11212	Chloe Campbell	17	17	100.00
11215	Ana Perry	17	17	100.00
11223	Hailey Patterson	27	27	100.00
11241	Lisa Cai	7	7	100.00
11242	Larry Munoz	7	7	100.00
11253	José Hernandez	16	16	100.00
11262	Jennifer Simmons	27	27	100.00
11276	Nancy Chapman	27	27	100.00
11277	Charles Jackson	27	27	100.00
11287	Henry Garcia	27	27	100.00
11019	Luke Lal	17	17	100.00
"""


# --- 4. Definición del Único Nodo del Subgrafo ---

In [38]:
def analizar_resultados_node(state: SubGraphState) -> dict:
    """
    Nodo que analiza los resultados de la consulta SQL y genera conclusiones y HTML.
    Utiliza el prompt y la lógica exactos del notebook original.
    """
    print(">> Entrando en el nodo: analizar_resultados_node")
    sql_query = state['sql_query']
    resultados_tabla = state['sql_results']
    
    # El prompt es una copia exacta del proporcionado en el notebook.
    prompt = f"""
    Eres un analista experto en el dominio de la base de datos consultada.

    La consulta realizada es:

    {sql_query}

    Recibirás la salida en texto plano de una consulta SQL:
    
    {resultados_tabla}

    
    Tu tarea es extraer las conclusiones principales de los datos, redactadas de forma breve y directa, sin explicaciones 
    innecesarias ni contexto adicional.
    Dirígete a un usuario con conocimientos en el tema, por lo que no es necesario definir conceptos básicos.
    Máximo 6 frases concisas, usando terminología técnica del dominio cuando corresponda.
    No incluyas datos sin relevancia ni interpretaciones especulativas.
    Así mismo, con aquellos datos que consideres más importantes, debes crear un códifo HTML en el que se mustren un máximo de 3
    gráficos con la información más importante para el negocio según tu criterio.
    """

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )
    
    analisis_completo = response.choices[0].message.content.strip()
    print("   - Análisis y HTML generados.")
    return {"final_analysis": analisis_completo}

# --- 5. Construcción del Subgrafo ---


In [ ]:
workflow = StateGraph(SubGraphState)

# Añadir el único nodo al grafo
workflow.add_node("analizar_resultados", analizar_resultados_node)

# Definir el flujo: el nodo de análisis es el punto de entrada y de salida.
workflow.set_entry_point("analizar_resultados")
workflow.add_edge("analizar_resultados", END)

# Compilar el grafo para obtener el subgrafo ejecutable
subgraph_analisis_sql = workflow.compile()

# --- 6. Invocación del Subgrafo ---
# Un grafo organizador superior le pasaría los datos de la consulta y sus resultados.
print("--- INICIANDO EJECUCIÓN DEL SUBGRAFO ---")
final_state = subgraph_analisis_sql.invoke({
    "sql_query": fixed_sql_query,
    "sql_results": fixed_sql_results
})
print("--- FINALIZANDO EJECUCIÓN DEL SUBGRAFO ---\n")

# El resultado final está en la clave 'final_analysis' del estado devuelto.
print("--- ANÁLISIS FINAL GENERADO ---")
print(final_state['final_analysis'])

# Opcionalmente, guardar el HTML en un archivo para visualizarlo.
# Se busca el bloque de código HTML en la respuesta para extraerlo.
try:

    # Expresión regular: captura desde "### Conclusiones principales:" hasta antes de "### Código HTML"
    patron = r"(### Conclusiones principales:[\s\S]*?)(?=### Código HTML)"
    resultado = re.search(patron, final_state['final_analysis'])

    if resultado:
        conclusiones = resultado.group(1).strip()
                
    with open("analisis_promociones.txt", "w", encoding="utf-8") as f:
        f.write(conclusiones)
    
    html_content = final_state['final_analysis'].split("### Conclusiones principales:")[1].split("###")[0]
    with open("analisis_clientes_promocion.html", "w", encoding="utf-8") as e:
        e.write(html_content.strip())
    print("\n\nEl archivo 'analisis_clientes_promocion.html' ha sido creado. Ábrelo en un navegador para ver los gráficos.")
except IndexError:
    print("\n\nNo se encontró un bloque de código HTML para guardar en el archivo.")


--- INICIANDO EJECUCIÓN DEL SUBGRAFO ---
>> Entrando en el nodo: analizar_resultados_node
   - Análisis y HTML generados.
--- FINALIZANDO EJECUCIÓN DEL SUBGRAFO ---

--- ANÁLISIS FINAL GENERADO ---
### Conclusiones principales:

1. Todos los clientes en la salida tienen una tasa de conversión de promoción del 100%, indicando que cada pedido realizado incluyó una promoción.
2. Los clientes con mayor cantidad de pedidos totales son Dalton Perez y Mason Roberts, ambos con 28 pedidos.
3. La mayoría de los clientes tienen entre 5 y 28 pedidos totales, cumpliendo el criterio de al menos 5 pedidos.
4. Los clientes con menor cantidad de pedidos totales en la lista son Amanda Rivera, Lisa Cai y Larry Munoz, cada uno con 5 y 7 pedidos respectivamente.
5. La promoción parece ser un factor determinante en la decisión de compra para estos clientes, dado que todos los pedidos incluyeron promociones.
6. Los datos sugieren que las promociones tienen un impacto significativo en la conversión de ventas 

In [47]:
final_state

{'sql_query': "\nSELECT\n    dc.customer_key,\n    dc.first_name || ' ' || dc.last_name AS cliente,\n    COUNT(DISTINCT fis.sales_order_number) AS pedidos_totales,\n    COUNT(DISTINCT CASE WHEN dpromo.promotion_key IS NOT NULL THEN fis.sales_order_number END) AS pedidos_con_promo,\n    ROUND(\n        100.0 * COUNT(DISTINCT CASE WHEN dpromo.promotion_key IS NOT NULL THEN fis.sales_order_number END) \n        / NULLIF(COUNT(DISTINCT fis.sales_order_number), 0),\n        2\n    ) AS tasa_conversion_promocion_pct\nFROM adventure_works.fact_internet_sales fis\nJOIN adventure_works.dim_customer dc\n    ON fis.customer_key = dc.customer_key\nLEFT JOIN adventure_works.dim_promotion dpromo\n    ON fis.promotion_key = dpromo.promotion_key\nWHERE fis.order_date BETWEEN '2004-01-01' AND '2020-12-31'\nGROUP BY dc.customer_key, cliente\nHAVING COUNT(DISTINCT fis.sales_order_number) >= 5\nORDER BY tasa_conversion_promocion_pct DESC\nLIMIT 20;\n",
 'sql_results': 'customer_key\tcliente\tpedidos_total

# --- 5. Simulación de la Invocación desde un Grafo Organizador ---


In [ ]:
"""
# Datos de entrada que llegarían al subgrafo
user_query_ejemplo = "Muestrame el top 20 de clientes con la mayor tasa de conversión en promociones, que tengan al menos 5 pedidos entre 2004 y 2020."
schema_summary_ejemplo = """
#- Tabla `fact_internet_sales` (alias fis): Contiene datos de ventas por internet. Columnas clave: `sales_order_number`, `customer_key`, `promotion_key`, `order_date`.
#- Tabla `dim_customer` (alias dc): Contiene datos de los clientes. Columnas clave: `customer_key`, `first_name`, `last_name`.
#- Tabla `dim_promotion` (alias dpromo): Contiene datos sobre las promociones. Columnas clave: `promotion_key`.
"""

# El grafo "organizador" invocaría al subgrafo de esta manera:
print("--- INICIANDO EJECUCIÓN DEL SUBGRAFO ---")
final_state = subgraph_analisis_ventas.invoke({
    "user_query": user_query_ejemplo,
    "schema_summary": schema_summary_ejemplo
})
print("--- FINALIZANDO EJECUCIÓN DEL SUBGRAFO ---\n")

# El resultado final está en la clave 'final_analysis' del estado devuelto
print("--- ANÁLISIS FINAL GENERADO ---")
print(final_state['final_analysis'])

txt_content = final_state['final_analysis']
with open("analisis_promociones.httxtml", "w", encoding="utf-8") as f:
    f.write(txt_content)


# Opcionalmente, puedes guardar el HTML en un archivo para visualizarlo
html_content = final_state['final_analysis'].split("```html\n")[1].split("```")[0]
with open("analisis_promociones.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("\n\nEl archivo 'analisis_promociones.html' ha sido creado. Ábrelo en un navegador para ver los gráficos.")


"""

'\n\n# El grafo "organizador" invocaría al subgrafo de esta manera:\nprint("--- INICIANDO EJECUCIÓN DEL SUBGRAFO ---")\nfinal_state = subgraph_analisis_ventas.invoke({\n    "user_query": user_query_ejemplo,\n    "schema_summary": schema_summary_ejemplo\n})\nprint("--- FINALIZANDO EJECUCIÓN DEL SUBGRAFO ---\n")\n\n# El resultado final está en la clave \'final_analysis\' del estado devuelto\nprint("--- ANÁLISIS FINAL GENERADO ---")\nprint(final_state[\'final_analysis\'])\n\ntxt_content = final_state[\'final_analysis\']\nwith open("analisis_promociones.httxtml", "w", encoding="utf-8") as f:\n    f.write(txt_content)\n\n\n# Opcionalmente, puedes guardar el HTML en un archivo para visualizarlo\nhtml_content = final_state[\'final_analysis\'].split("```html\n")[1].split("```")[0]\nwith open("analisis_promociones.html", "w", encoding="utf-8") as f:\n    f.write(html_content)\n\nprint("\n\nEl archivo \'analisis_promociones.html\' ha sido creado. Ábrelo en un navegador para ver los gráficos.")\n